# ISIC 2024 - Skin Cancer Detection: Pytorch Model w/ Image + Metadata (5 Folds)

Inspired by [motono0223](https://www.kaggle.com/code/motono0223/isic-pytorch-training-baseline-eva02/notebook#Run-Training) 

Idea:
* Use 1:20 benign vs malignant training data
* Feature engineer and append snippet of data to images
* Data augment images for diversity
* Train Vision Transformer and simple PyTorch model
* Save model base on better validation AUROC

Notebooks:
* Training notebook (current)
* [Inference notebook](https://www.kaggle.com/code/qiaoyingzhang/isic-2024-pytorch-inference-swin-vit/notebook)

# Import Libraries

In [ ]:
!pip install torcheval

In [ ]:
import os
import gc
import cv2
import copy
import time
import random
import glob
import h5py
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image
from io import BytesIO

# PyTorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torcheval.metrics.functional import binary_auroc

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import StratifiedGroupKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

# Initialize Environment & Configuration

In [ ]:
CONFIG = {
    "seed": 2024,
    "epochs": 50,
    "img_size": 224,
    "model_name": 'vit_base_patch16_224',
    "train_batch_size": 32,
    "valid_batch_size": 64,
    "learning_rate": 1e-5,
    "scheduler": 'CosineAnnealingLR',
    "min_lr": 1e-10,
    "T_max": 500,
    "weight_decay": 1e-6,
#     "fold" : 1,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
}

# Set seed for reproducibility
def seed_everything(seed):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)
    
seed_everything(CONFIG['seed'])

In [ ]:
ROOT_DIR = "/kaggle/input/isic-2024-challenge"
HDF_FILE = f"{ROOT_DIR}/train-image.hdf5"

# Data Preparation

In [ ]:
df = pd.read_csv(f"{ROOT_DIR}/train-metadata.csv")

print("       df.shape, # of positive cases, # of patients")
print("original>", df.shape, df.target.sum(), df["patient_id"].unique().shape)

df_positive = df[df["target"] == 1].reset_index(drop=True)
df_negative = df[df["target"] == 0].reset_index(drop=True)

# Data ratio -> positive:negative = 1:25
df = pd.concat([df_positive, df_negative.iloc[:df_positive.shape[0]*20, :]])  
print("filtered>", df.shape, df.target.sum(), df["patient_id"].unique().shape)

df = df.reset_index(drop=True)
print("df.shape, # of positive cases, # of patients")
print("original>", df.shape, df.target.sum(), df["patient_id"].unique().shape)

In [ ]:
# Feature engineering
df['lesion_size_ratio'] = df['tbp_lv_minorAxisMM'] / df['clin_size_long_diam_mm']
df['color_uniformity'] = df['tbp_lv_color_std_mean'] / df['tbp_lv_radial_color_std_max']
df['3d_position_distance'] = np.sqrt(df['tbp_lv_x'] ** 2 + df['tbp_lv_y'] ** 2 + df['tbp_lv_z'] ** 2) 
# List of numerical features to be scaled
num_feat = ['age_approx', 'lesion_size_ratio', 'color_uniformity', 
            'tbp_lv_Lext', 'tbp_lv_eccentricity', '3d_position_distance']

# Replace infinite values with NaN
df[num_feat] = df[num_feat].replace([np.inf, -np.inf], np.nan)

# Handle missing values (if any) by filling them with the mean of the column
df[num_feat] = df[num_feat].fillna(df[num_feat].mean())

# Scale numerical features
scaler = StandardScaler()
df[num_feat] = scaler.fit_transform(df[num_feat])

print("Feature engineering and scaling complete.")

In [ ]:
CONFIG['T_max'] = df.shape[0] * (CONFIG['n_fold']-1) * CONFIG['epochs'] \
// CONFIG['train_batch_size'] // CONFIG['n_fold']

print(CONFIG['T_max'])

# Dataset Manipulation

In [ ]:
# Dataset
class ISICDataset(Dataset):
    def __init__(self, dataframe, feat, file_path, transforms=None, is_training=True):
        self.df = dataframe
        self.file_path = h5py.File(file_path, mode="r")
        self.transforms = transforms
        self.is_training = is_training
        self.df_positive = self.df[self.df['target'] == 1].reset_index()
        self.df_negative = self.df[self.df['target'] == 0].reset_index()
        self.isic_ids_positive = self.df_positive['isic_id'].values
        self.isic_ids_negative = self.df_negative['isic_id'].values
        self.targets_positive = self.df_positive['target'].values
        self.targets_negative = self.df_negative['target'].values
        self.metadata = self.df[feat].values

    def __len__(self):
        return len(self.df_positive) * 2 if self.is_training else len(self.df)

    def __getitem__(self, idx):
        if self.is_training:
            is_positive = random.random() >= 0.5
            df_subset = self.df_positive if is_positive else self.df_negative
            isic_ids = self.isic_ids_positive if is_positive else self.isic_ids_negative
            targets = self.targets_positive if is_positive else self.targets_negative
            idx = idx % len(df_subset)
            target = targets[idx]
            isic_id = isic_ids[idx]
        else:
            row = self.df.iloc[idx]
            target = row['target']
            isic_id = row['isic_id']
        
        metadata = self.metadata[idx]
        
        image = np.array(Image.open(BytesIO(self.file_path[isic_id][()])))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transforms:
            image = self.transforms(image=image)["image"]
        
        return {'image': image, 'target': target, 'metadata': metadata}

# Modeling

In [ ]:
# Modeling - hybrid model w/ CNN + Metadata
class CustomISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=True):
        super(CustomISICModel, self).__init__()
        self.image_model = timm.create_model(model_name, pretrained=pretrained)
        self.image_out_features = self.image_model.get_classifier().in_features
        self.image_model.reset_classifier(0)  # Remove the original classifier

        # Metadata part
        metadata_input_features = 6
        metadata_output_features = 128

        self.metadata_fc = nn.Sequential(
            nn.Linear(metadata_input_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, metadata_output_features),
            nn.BatchNorm1d(metadata_output_features),
            nn.ReLU()
        )

        # Combine features from image model and metadata
        combined_features = self.image_out_features + metadata_output_features
        self.final_fc = nn.Sequential(
            nn.Linear(combined_features, 512),
            nn.BatchNorm1d(512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
            nn.Sigmoid()
        )

    def forward(self, image, metadata):
        image_features = self.image_model(image)
        metadata_features = self.metadata_fc(metadata)
        combined_features = torch.cat((image_features, metadata_features), dim=1)
        output = self.final_fc(combined_features)
        
        return output

# Check model    
model = CustomISICModel(CONFIG['model_name'], pretrained=True)

model.to(CONFIG['device'])

# Data Augmentation & Helper Functions

In [ ]:
# Define image transformer
data_transforms = {
    'train': A.Compose([
        A.RandomRotate90(p=0.5),
        A.Flip(p=0.5),
        A.Transpose(p=0.5),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=45, p=0.5),
        A.OneOf([
            A.HueSaturationValue(p=0.5),
            A.RandomBrightnessContrast(p=0.5),
        ], p=0.5),
        A.OneOf([
            A.GaussNoise(p=0.5),
            A.GaussianBlur(p=0.5),
            A.MotionBlur(p=0.5),
        ], p=0.5),
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
            mean=[0.4815, 0.4578, 0.4082], 
            std=[0.2686, 0.2613, 0.2758], 
            max_pixel_value=255.0),
        ToTensorV2(),
    ]),
    
    'valid': A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(
            mean=[0.4815, 0.4578, 0.4082], 
            std=[0.2686, 0.2613, 0.2758], 
            max_pixel_value=255.0),
        ToTensorV2(),
    ])
}

In [ ]:
# Loss function
def criterion(outputs, targets):
    return nn.BCELoss()(outputs, targets)

In [ ]:
# Running training and validation
def prepare_loaders(df, fold):
    train_df = df[df['fold'] != fold].reset_index(drop=True)
    valid_df = df[df['fold'] == fold].reset_index(drop=True)

    train_dataset = ISICDataset(train_df, feat=num_feat, file_path=HDF_FILE, 
                                transforms=data_transforms['train'], is_training=True)
    valid_dataset = ISICDataset(valid_df, feat=num_feat, file_path=HDF_FILE, 
                                transforms=data_transforms['valid'], is_training=False)

    train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'], shuffle=True, 
                              num_workers=4, pin_memory=True)
    valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'], shuffle=False, 
                              num_workers=4, pin_memory=True)
    
    return train_loader, valid_loader

# Training & Validation Functions

In [ ]:
# Main training function
def run_training(model, optimizer, scheduler, device, num_epochs, fold):
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = 0
    best_epoch_loss = 1
    history = defaultdict(list)
    
    for epoch in range(1, num_epochs + 1):
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, 
                                                              scheduler, dataloader=train_loader, 
                                                              device=device, epoch=epoch)
        
        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, valid_loader, device=device, epoch=epoch)
        
        history['Train Loss'].append(train_epoch_loss)
        history['Train AUROC'].append(train_epoch_auroc)
        history['Valid Loss'].append(val_epoch_loss)
        history['Valid AUROC'].append(val_epoch_auroc)
        
        # deep copy the model (balance auroc and loss accuracy)
        if (best_epoch_auroc <= val_epoch_auroc) & (best_epoch_loss - val_epoch_loss >= -0.05):
            print(f"{b_}Validation AUROC Improved ({best_epoch_auroc} ---> {val_epoch_auroc})")
            best_epoch_auroc = val_epoch_auroc
            best_epoch_loss = val_epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            
            # Save a model file from the current directory
            PATH = f"best_AUROC_model_fold{fold}.bin"
            torch.save(model.state_dict(), PATH)
            print(f"Model Saved -- epoch: {epoch:.0f}, AUROC: {val_epoch_auroc:.4f}, Loss: {val_epoch_loss:.4f}{sr_}")

    print('Best val AUROC: {:4f}'.format(best_epoch_auroc))
    
    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, history

In [ ]:
# Training
def train_one_epoch(model, optimizer, scheduler, dataloader, device, epoch):
    model.train()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        metadata = data['metadata'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images, metadata).squeeze()
        loss = criterion(outputs, targets)
        loss = loss / CONFIG['n_accumulate']
            
        loss.backward()
    
        if (step + 1) % CONFIG['n_accumulate'] == 0:
            optimizer.step()

            # zero the parameter gradients
            optimizer.zero_grad()

            if scheduler is not None:
                scheduler.step()
                
        auroc = binary_auroc(input=outputs, target=targets).item()
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size
        
        epoch_loss = running_loss / dataset_size
        epoch_auroc  = running_auroc  / dataset_size
        
        bar.set_postfix(Epoch=epoch, Train_Loss=epoch_loss, Train_AUROC=epoch_auroc, 
                        LR=optimizer.param_groups[0]['lr'])
        
    gc.collect()
    
    return epoch_loss, epoch_auroc 

In [ ]:
# Validation

@torch.no_grad()
def valid_one_epoch(model, dataloader, device, epoch):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    running_auroc  = 0.0
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        images = data['image'].to(device, dtype=torch.float)
        metadata = data['metadata'].to(device, dtype=torch.float)
        targets = data['target'].to(device, dtype=torch.float)
        
        batch_size = images.size(0)
        
        outputs = model(images, metadata).squeeze()
        loss = criterion(outputs, targets)
        
        auroc = binary_auroc(input=outputs, target=targets).item()
        
        running_loss += (loss.item() * batch_size)
        running_auroc  += (auroc * batch_size)
        dataset_size += batch_size

        epoch_loss = running_loss / dataset_size
        epoch_auroc  = running_auroc  / dataset_size
        
        bar.set_postfix(Epoch=epoch, Valid_Loss=epoch_loss, Valid_AUROC=epoch_auroc)
        
    gc.collect()
    
    return epoch_loss, epoch_auroc 

# Training

In [ ]:
# Plot Loss and AUROC Curves
def plot_metrics(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    ax1.plot(history['Train Loss'], label='Train Loss')
    ax1.plot(history['Valid Loss'], label='Valid Loss')
    ax1.set_title('Loss Curves')
    ax1.legend()

    ax2.plot(history['Train AUROC'], label='Train AUROC')
    ax2.plot(history['Valid AUROC'], label='Valid AUROC')
    ax2.set_title('AUROC Curves')
    ax2.legend()

    plt.show()

# plot_metrics(history)

In [ ]:
# Preparing stratified k-fold
skf = StratifiedGroupKFold(n_splits=CONFIG['n_fold'])

for fold, (train_idx, val_idx) in enumerate(skf.split(X=df, y=df.target, groups=df.patient_id)):
    df.loc[val_idx, 'fold'] = fold

In [ ]:
# Get best model for multiple folds
for fold in range(CONFIG['n_fold']):
    print(f'{b_}Training Fold {fold}')
    print(f'------------------------------{sr_}')
    torch.cuda.empty_cache()
    
    # Setup model
    model = CustomISICModel(CONFIG['model_name'], pretrained=True)
    model.to(CONFIG['device'])
    
    # Optimizer & Scheduler
    optimizer = optim.Adam(model.parameters(), lr=CONFIG['learning_rate'], 
                           weight_decay=CONFIG['weight_decay'])

    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['T_max'], 
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=CONFIG['T_0'], 
                                                             eta_min=CONFIG['min_lr'])
        
    # Prepare Dataloaders
    train_loader, valid_loader = prepare_loaders(df, fold=fold)
    
    # Run training
    model, history = run_training(model, optimizer, scheduler,
                                  CONFIG['device'], CONFIG['epochs'], fold)
    
    # Metric logs
    plot_metrics(history)